### Modules and other utils

In [1]:
import numpy as np
import pandas as pd
import opendatasets as od
from sklearn.metrics.pairwise import cosine_similarity as c_score

od.download("https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset")

ex = "Harry Potter and the Prisoner of Azkaban"
cols = ['Book-Title', 'Book-Author', 'Image-URL-M', 'Votes', 'Avg-rating']

def df_info(df):
    print(f"{df.head()}\n\nShape\t:\t{df.shape}")

Skipping, found downloaded files in ".\book-recommendation-dataset" (use force=True to force download)


### Importing the **datasets**

In [2]:
books = pd.read_csv('book-recommendation-dataset/Books.csv')
users = pd.read_csv('book-recommendation-dataset/Users.csv')
ratings = pd.read_csv('book-recommendation-dataset/Ratings.csv')

C:\Users\mishr\AppData\Local\Temp\ipykernel_9156\2205890256.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('book-recommendation-dataset/Books.csv')


### Fixing Image URLs

In [3]:
books['Image-URL-M'] = books['Image-URL-M'].str.replace('http', 'https')

## **Popularity-based** Recommendation System

In [4]:
temp_br = books.merge(ratings, on="ISBN")
temp_br.shape

(1031136, 10)

### Number of Ratings

In [5]:
temp_num = temp_br.groupby('Book-Title').count()['Book-Rating'].reset_index()
temp_num.rename(columns={'Book-Rating': 'Votes'}, inplace=True)
temp_num

,Book-Title,Votes
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


### Average Rating

In [6]:
temp_avg = temp_br.groupby('Book-Title')['Book-Rating'].mean().reset_index()
temp_avg.rename(columns={'Book-Rating': 'Avg-rating'}, inplace=True)
temp_avg

,Book-Title,Avg-rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


### Popular books  -  based on `Avg-rating`

In [7]:
pop_ = temp_num.merge(temp_avg, on='Book-Title')
pop_

,Book-Title,Votes,Avg-rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


### Filtering books based on `Votes`
Obtaining **top 50** books based on `Votes >= 250`

In [8]:
temp__ = pop_[pop_['Votes'] >= 250].sort_values('Avg-rating', ascending=False)
pop = temp__.head(50)
pop.shape

(50, 3)

### Merging with `books` to obtain more data

In [9]:
top50 = pop.merge(books, on='Book-Title')[cols]
top50.drop_duplicates('Book-Title', inplace=True)
top50

,Book-Title,Book-Author,Image-URL-M,Votes,Avg-rating
0,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,https://images.amazon.com/images/P/0439136350....,428,5.852804
3,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,https://images.amazon.com/images/P/0439139597....,387,5.824289
5,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,https://images.amazon.com/images/P/0590353403....,278,5.737410
9,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,https://images.amazon.com/images/P/043935806X....,347,5.501441
13,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,https://images.amazon.com/images/P/0439064872....,556,5.183453
16,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,https://images.amazon.com/images/P/0345339681....,281,5.007117
17,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. TOLKIEN,https://images.amazon.com/images/P/0345339703....,368,4.948370
26,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,https://images.amazon.com/images/P/059035342X....,575,4.895652
28,"The Two Towers (The Lord of the Rings, Part 2)",J.R.R. TOLKIEN,https://images.amazon.com/images/P/0345339711....,260,4.880769
39,To Kill a Mockingbird,Harper Lee,https://images.amazon.com/images/P/0446310786....,510,4.700000


### Cleaning the `top50`

In [10]:
top50['Avg-rating'] = round(top50['Avg-rating'], 2)
top50['Book-Title'] = top50['Book-Title'].str.strip().replace(r'\s{1,}\(.*\)', '', regex=True)
top50.head()

,Book-Title,Book-Author,Image-URL-M,Votes,Avg-rating
0,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,https://images.amazon.com/images/P/0439136350....,428,5.85
3,Harry Potter and the Goblet of Fire,J. K. Rowling,https://images.amazon.com/images/P/0439139597....,387,5.82
5,Harry Potter and the Sorcerer's Stone,J. K. Rowling,https://images.amazon.com/images/P/0590353403....,278,5.74
9,Harry Potter and the Order of the Phoenix,J. K. Rowling,https://images.amazon.com/images/P/043935806X....,347,5.50
13,Harry Potter and the Chamber of Secrets,J. K. Rowling,https://images.amazon.com/images/P/0439064872....,556,5.18


### Outputting the `top50` dataset

In [11]:
top50.to_csv('processed-dataset/top50.csv')

## User-based **Collaborative Filtering**

### Obtaining `User-ID` with **more than 200** reviews

In [12]:
x = temp_br.groupby('User-ID').count()['Book-Rating']
top_users = x[x > 200].index
top_users

Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,   6323,
         6543,
       ...
       271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427, 277639,
       278418],
      dtype='int64', name='User-ID', length=811)

### Obtaining the records made by `top_users`

In [13]:
filtered_users = temp_br[temp_br['User-ID'].isin(top_users)]
df_info(filtered_users)

          ISBN                                         Book-Title  \
3   0002005018                                       Clara Callan   
6   0002005018                                       Clara Callan   
7   0002005018                                       Clara Callan   
10  0002005018                                       Clara Callan   
21  0374157065  Flu: The Story of the Great Influenza Pandemic...   

             Book-Author Year-Of-Publication              Publisher  \
3   Richard Bruce Wright                2001  HarperFlamingo Canada   
6   Richard Bruce Wright                2001  HarperFlamingo Canada   
7   Richard Bruce Wright                2001  HarperFlamingo Canada   
10  Richard Bruce Wright                2001  HarperFlamingo Canada   
21      Gina Bari Kolata                1999   Farrar Straus Giroux   

                                          Image-URL-S  \
3   http://images.amazon.com/images/P/0002005018.0...   
6   http://images.amazon.com/images/P/000200

### Obtaining the `Book-Title` with **more than 50** ratings

In [14]:
y = filtered_users.groupby('Book-Title').count()['Book-Rating']
famous_books = y[y >= 50].reset_index()['Book-Title'].values

### Obtaining the `famous_books` records

In [15]:
filtered_books = filtered_users[filtered_users['Book-Title'].isin(famous_books)]
filtered_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,https://images.amazon.com/images/P/0399135782....,http://images.amazon.com/images/P/0399135782.0...,11676,9
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,https://images.amazon.com/images/P/0399135782....,http://images.amazon.com/images/P/0399135782.0...,36836,0
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,https://images.amazon.com/images/P/0399135782....,http://images.amazon.com/images/P/0399135782.0...,46398,9
38,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,https://images.amazon.com/images/P/0399135782....,http://images.amazon.com/images/P/0399135782.0...,113270,0
39,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,https://images.amazon.com/images/P/0399135782....,http://images.amazon.com/images/P/0399135782.0...,113519,0


### Cleaning the `filtered books` book titles

In [16]:
filtered_books['Book-Title'] = filtered_books['Book-Title'].str.strip().replace(r'\s{1,}\(.*\)', '', regex=True)
filtered_books['Book-Title'] = filtered_books['Book-Title'].str.replace('&amp;', 'and')
filtered_books['Book-Title'] = filtered_books['Book-Title'].str.replace('\\O\\" Is for Outlaw"', "O is for Outlaw")
filtered_books[filtered_books['Book-Title'] == ex]

C:\Users\mishr\AppData\Local\Temp\ipykernel_9156\974687905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Book-Title'] = filtered_books['Book-Title'].str.strip().replace(r'\s{1,}\(.*\)', '', regex=True)
C:\Users\mishr\AppData\Local\Temp\ipykernel_9156\974687905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_books['Book-Title'] = filtered_books['Book-Title'].str.replace('&amp;', 'and')
C:\Users\mishr\AppData\Local\Temp\ipykernel_9156\974687905.py:3: SettingWithCopyWarning: 

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
142493,0439136350,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,https://images.amazon.com/images/P/0439136350....,http://images.amazon.com/images/P/0439136350.0...,254,9
142499,0439136350,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,https://images.amazon.com/images/P/0439136350....,http://images.amazon.com/images/P/0439136350.0...,6251,10
142501,0439136350,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,https://images.amazon.com/images/P/0439136350....,http://images.amazon.com/images/P/0439136350.0...,8681,0
142503,0439136350,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,https://images.amazon.com/images/P/0439136350....,http://images.amazon.com/images/P/0439136350.0...,11676,0
142505,0439136350,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,https://images.amazon.com/images/P/0439136350....,http://images.amazon.com/images/P/0439136350.0...,16106,0
...,...,...,...,...,...,...,...,...,...,...
201660,0439136369,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,https://images.amazon.com/images/P/0439136369....,http://images.amazon.com/images/P/0439136369.0...,265313,9
201665,0439136369,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,https://images.amazon.com/images/P/0439136369....,http://images.amazon.com/images/P/0439136369.0...,271448,0
201667,0439136369,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,https://images.amazon.com/images/P/0439136369....,http://images.amazon.com/images/P/0439136369.0...,273979,0
714163,0786222743,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,2000,Thorndike Press,http://images.amazon.com/images/P/0786222743.0...,https://images.amazon.com/images/P/0786222743....,http://images.amazon.com/images/P/0786222743.0...,11676,6


### **Pivoting** the table `filtered_books`
On the `Book-Title` column as **index**, with `User-ID` as **columns** and `Book-Rating` as **values**

In [17]:
pt = filtered_books.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0.0)
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wuthering Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Computing the **cosine similarity** scores

In [18]:
sim_scores = c_score(pt)
sim_scores, sim_scores.shape

(array([[1.        , 0.10255025, 0.01220856, ..., 0.01769644, 0.12110367,
         0.07347567],
        [0.10255025, 1.        , 0.2364573 , ..., 0.19309722, 0.07446129,
         0.16773875],
        [0.01220856, 0.2364573 , 1.        , ..., 0.16565383, 0.04558758,
         0.04938579],
        ...,
        [0.01769644, 0.19309722, 0.16565383, ..., 1.        , 0.0521682 ,
         0.19430288],
        [0.12110367, 0.07446129, 0.04558758, ..., 0.0521682 , 1.        ,
         0.07085128],
        [0.07347567, 0.16773875, 0.04938579, ..., 0.19430288, 0.07085128,
         1.        ]]),
 (698, 698))

### **Recommender** function

In [19]:
def recommend(book):
    suggestions = []
    index = np.where(pt.index == book)[0][0]
    s_books = sorted(list(enumerate(sim_scores[index])), key=lambda x:x[1], reverse=True)[1:6]
    for book in s_books:
        suggestions.append(pt.index[book[0]])
    return suggestions

### Obtaining recommended books i.e. **top 5** suggestions

In [29]:
recommend("Harry Potter and the Sorcerer's Stone")

['Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Order of the Phoenix',
 'The Mists of Avalon']

### Rough Work

#### Enumerated list of `SIMILARITY SCORES` i.e. each index represents the corres. index in `pt` table

In [21]:
s = list(enumerate(sim_scores))
s[0][1]                        # will give the SIMILARITY SCORE of `1984` against ALL the 698 books in pt

array([1.        , 0.10255025, 0.01220856, 0.        , 0.05367224,
       0.02774901, 0.08216491, 0.13732869, 0.03261686, 0.03667591,
       0.02322418, 0.06766487, 0.02083978, 0.09673735, 0.13388865,
       0.08303112, 0.11153543, 0.05100411, 0.02517784, 0.11706383,
       0.        , 0.14333793, 0.07847534, 0.06150451, 0.08723968,
       0.        , 0.07009814, 0.13658681, 0.07600328, 0.12167134,
       0.00768046, 0.01473221, 0.        , 0.07965814, 0.04522617,
       0.01556271, 0.09495938, 0.0182307 , 0.02610465, 0.07984012,
       0.11679969, 0.0569124 , 0.08354155, 0.08471898, 0.08785938,
       0.05491435, 0.0548505 , 0.27026514, 0.09779123, 0.06016046,
       0.08958835, 0.06748675, 0.        , 0.04468098, 0.01920872,
       0.        , 0.05629067, 0.00557964, 0.07877059, 0.05219479,
       0.18908177, 0.        , 0.01240656, 0.02984572, 0.04279502,
       0.19591539, 0.        , 0.13357242, 0.06615478, 0.        ,
       0.        , 0.        , 0.10968075, 0.02806606, 0.04521

#### Fetching the `index` of each book

In [22]:
print(f"Index of 'The Da Vinci Code' : {np.where(pt.index == ex)[0][0]}")

Index of 'The Da Vinci Code' : 228


#### Sorting the `SIMILARITY SCORES` in **descending order**

In [23]:
for p, q in enumerate(s[0][1]):
    print(f"Book# : {p} - SCORE : {q}") # will display all the 698 books' SIMILARITY SCORES with index on LHS against '1984'

sample = list(enumerate(sim_scores[0])) # wil sort the scores against book '1984' in reverse order
z = sorted(sample, key=lambda x:x[1], reverse=True)[1:6]

Book# : 0 - SCORE : 0.9999999999999994
Book# : 1 - SCORE : 0.10255024829874465
Book# : 2 - SCORE : 0.012208555646509366
Book# : 3 - SCORE : 0.0
Book# : 4 - SCORE : 0.053672244225442485
Book# : 5 - SCORE : 0.027749013024662858
Book# : 6 - SCORE : 0.08216491130000354
Book# : 7 - SCORE : 0.13732869060263378
Book# : 8 - SCORE : 0.03261686350494277
Book# : 9 - SCORE : 0.03667590746512177
Book# : 10 - SCORE : 0.02322418425912186
Book# : 11 - SCORE : 0.06766486753662343
Book# : 12 - SCORE : 0.020839782065795794
Book# : 13 - SCORE : 0.09673734976610586
Book# : 14 - SCORE : 0.13388864701187034
Book# : 15 - SCORE : 0.08303111691811028
Book# : 16 - SCORE : 0.1115354275858341
Book# : 17 - SCORE : 0.051004105100675136
Book# : 18 - SCORE : 0.025177841632809915
Book# : 19 - SCORE : 0.11706383185253948
Book# : 20 - SCORE : 0.0
Book# : 21 - SCORE : 0.14333792815336935
Book# : 22 - SCORE : 0.07847534327232127
Book# : 23 - SCORE : 0.06150451339525138
Book# : 24 - SCORE : 0.08723968326958671
Book# : 25 - 

#### Fetching the `Book-Title` using the **index** of `pt`  table

In [24]:
ctr = 0
for index, book_score in z:
    if ctr!=4:
        print(f"Book Name : {pt.index[index]}\nScore : {book_score}\n")
        ctr+=1
    else:
        print(f"Book Name : {pt.index[index]}\nScore : {book_score}")

Book Name : Animal Farm
Score : 0.2702651417103732

Book Name : The Handmaid's Tale
Score : 0.26396193711234966

Book Name : Brave New World
Score : 0.2366937434740099

Book Name : The Vampire Lestat
Score : 0.23299389358170394

Book Name : The Hours : A Novel
Score : 0.2262639743141286


### Fetching all the details for each of the `suggestions`

#### Removing **duplicate names** from `filtered_books`
Only `698` records remain

In [25]:
temp_books = filtered_books.drop_duplicates('Book-Title')[['Book-Title', 'Book-Author', 'Publisher', 'Year-Of-Publication', 'Image-URL-M']]
temp_books.to_csv('processed-dataset/final.csv')

#### **Top 5**  suggestions

In [26]:
all_suggestions = {}

for name in pt.index:
    all_suggestions.update({name: recommend(name)})
    
suggestions = pd.DataFrame(all_suggestions).T
suggestions.reset_index(inplace=True)
suggestions.rename(columns={'index': 'book-title', 0: "1st", 1: "2nd", 2: "3rd", 3: "4th", 4: "5th"}, inplace=True)

suggestions.to_csv('processed-dataset/sugg.csv')

#### Function for fetching details from `temp_books`

In [27]:
book_data = []
for name in suggestions[suggestions['book-title'] == ex].values[0][1:]:
    for data in temp_books[temp_books['Book-Title']==name].values:
        book_data.append(data)

for item in book_data:
    for i in item:
        print(i)
    print()

Harry Potter and the Goblet of Fire
J. K. Rowling
Scholastic
2000
https://images.amazon.com/images/P/0439139597.01.MZZZZZZZ.jpg

Harry Potter and the Chamber of Secrets
J. K. Rowling
Scholastic
2000
https://images.amazon.com/images/P/0439064872.01.MZZZZZZZ.jpg

Harry Potter and the Sorcerer's Stone
J. K. Rowling
Arthur A. Levine Books
1999
https://images.amazon.com/images/P/059035342X.01.MZZZZZZZ.jpg

Harry Potter and the Order of the Phoenix
J. K. Rowling
Scholastic
2003
https://images.amazon.com/images/P/043935806X.01.MZZZZZZZ.jpg

The Fellowship of the Ring
J.R.R. TOLKIEN
Del Rey
1986
https://images.amazon.com/images/P/0345339703.01.MZZZZZZZ.jpg

